## Import

In [1]:
pip install googletrans

Note: you may need to restart the kernel to use updated packages.


You are using pip version 18.1, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [25]:
import os
import tweepy as tw
import pandas as pd
import re

import json
import pickle
import time

#translate
from googletrans import Translator
translator = Translator(service_urls=['translate.google.com'])

# Twitter

## Tweepy credentials

In [2]:
consumer_key= 'enter here'
consumer_secret= 'enter here'
access_token= 'enter here'
access_token_secret= 'enter here'
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [3]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

## Search

In [4]:
search_words = "cpi"
date_since = "2019-01-16"

In [6]:
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="hi",
              since=date_since,tweet_mode="extended").items(2300)
l=[]
# Iterate and print tweets
for tweet in tweets:
    # print(re.sub("\n","",tweet.full_text))
    print("-",end="")
    try:
        l.append(re.sub("\n","",tweet.retweeted_status.full_text))
    except:
        l.append(re.sub("\n","",tweet.full_text))
l=list(set(l))

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
%%time
for i in range(len(l)):
    l[i] = re.sub("(@[A-Za-z0-9_]+)|(http\S+)|(#)|(&amp;)"," ",l[i])
    l[i] = re.sub("।",".",l[i])
    l[i] = deEmojify(l[i])

Wall time: 10.4 ms


In [13]:
len(l)

180

In [11]:
c=[]
for i in l:
    for j in i.split("."):
        if(j!=""):
            c.append(j)

In [27]:
%%time
final=[]
for q in c:
    try:
        final.append(translator.translate(q,src="hi",dest="en").text)
    except:
        pass

Wall time: 2min 21s


In [26]:
translator.translate("विधायक अमानतुल्ला खान तीसरी बार चुने गए दिल्ली वक्फ बोर्ड के अध्यक्ष ",src="hi",dest="en").text

'MLA Amanatullah Khan elected for the third time Chairman of Delhi Waqf Board'

In [29]:
with open("cpi_hi.txt", "wb") as fp:   #Pickling
      pickle.dump(final, fp)

In [19]:
with open('congress_en.txt',"rb") as fp:
    b = pickle.load(fp)

## Timeline


In [ ]:
public_tweets = api.home_timeline(tweet_mode="extended")
l=[]
for tweet in public_tweets:
    l.append(re.sub("\n","",tweet.full_text))
l